In [ ]:
import keras
import os
from skimage import io
import numpy as np
from keras import backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, DepthwiseConv2D
from keras.layers import Concatenate
from keras.models import load_model
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger
from keras.callbacks import Callback
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as k
import copy
import time

In [ ]:
class cifar10vgg:
    def __init__(self,train=True, ccp_augment=False):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model, ccp_augment)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        mean = np.mean(x)
        std = np.std(x)
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)


    def train(self,model, ccp_augment=False):

        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20

        def change_brightness(image, alpha, beta):
          new_image = np.zeros(image.shape, np.int64)
          new_image = np.clip(alpha*image, 0, 255)
          return new_image
          
        def CCP_Attack_Brightness(image, transform):
          img = copy.copy(image)
          for channel in range(img.shape[2]):
            temp1 = image[:,:,0]
            temp2 = image[:,:,1]
            temp3 = image[:,:,2]

            temp = temp1 * transform[channel][0] + temp2 * transform[channel][1] + temp3 * transform[channel][2]

            img[:,:,channel] = temp/3

          img1 = change_brightness(img, 2, 0)
          return img1

        (x_train, y_train), (x_test, y_test) = cifar10.load_data()

        if ccp_augment:
          x_train_trans = copy.copy(x_train)
          x_train_trans = x_train_trans.astype(np.int64)
          print('Transforming training dataset...')

          for test in range(len(x_train_trans)):
            a = np.random.uniform(low=0.0, high=1.0, size=(3,))
            b = np.random.uniform(low=0.0, high=1.0, size=(3,))
            c = np.random.uniform(low=0.0, high=1.0, size=(3,))
            transform = np.array([a,b,c])

            img = copy.copy(x_train[test])
            img = img.astype(np.int64)

            x_train_trans[test] = CCP_Attack_Brightness(img,transform)

          print('\nAugmented CCP Attacked Training Dataset!\n')


          x_train = np.vstack((x_train,x_train_trans))
          y_train = np.vstack((y_train,y_train))

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False, 
            featurewise_std_normalization=False,  
            samplewise_std_normalization=False, 
            zca_whitening=False,  
            rotation_range=15,  
            width_shift_range=0.1,  
            height_shift_range=0.1,  
            horizontal_flip=True,  
            vertical_flip=False)  
        datagen.fit(x_train)


        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        my_callbacks = [
            tf.keras.callbacks.ModelCheckpoint(filepath='cifar10vgg_{epoch:02d}_{val_loss:.2f}.h5', save_best_only=True, period=10),
            reduce_lr
        ]

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=175,
                            validation_data=(x_test, y_test),callbacks=my_callbacks,verbose=2)
        model.save('cifar10vgg.h5')
        return model


In [ ]:
model = cifar10vgg(train=True, ccp_augment=False) # Change ccp_augment=False to ccp_augment=True for CCP augmented training


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

predicted_x = model.predict(x_test)
residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

loss = sum(residuals)/len(residuals)
print("the validation 0/1 loss is: ",1-loss)

In [ ]:
def change_brightness(image, alpha, beta):
  new_image = np.zeros(image.shape, np.int64)
  new_image = np.clip( alpha*image + beta, 0, 255)
  return new_image
	
def CCP_Attack_Brightness(image, transform):
	img = copy.copy(image)
	for channel in range(img.shape[2]):
		temp1 = image[:,:,0]
		temp2 = image[:,:,1]
		temp3 = image[:,:,2]

		temp = temp1 * transform[channel][0] + temp2 * transform[channel][1] + temp3 * transform[channel][2]

		img[:,:,channel] = temp/3

	img1 = change_brightness(img, 2, 0)
	return img1

In [ ]:
(x_train1, y_train1), (x_test1, y_test1) = cifar10.load_data()

a = np.random.uniform(low=0.0, high=1.0, size=(3,))
b = np.random.uniform(low=0.0, high=1.0, size=(3,))
c = np.random.uniform(low=0.0, high=1.0, size=(3,))
transform1 = np.array([a,b,c])

x_test1_trans = copy.copy(x_test1)
x_test1_trans = x_test1_trans.astype(np.int64)

for test in range(len(x_test1_trans)):
  
  img = copy.copy(x_test1[test])
  img = img.astype(np.int64)

  x_test1_trans[test] = CCP_Attack_Brightness(img,transform1)


x_train1 = x_train1.astype('float32') / 255
x_test1_trans = x_test1_trans.astype('float32') / 255


y_train1 = keras.utils.to_categorical(y_train1, 10)
y_test1 = keras.utils.to_categorical(y_test1, 10)

predicted_x = model.predict(x_test1_trans)
residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)
loss = sum(residuals)/len(residuals)
print("the validation 0/1 loss (after CCP attack) is: ",1-loss)